# Attention on seq2seq
实现一个Attention在seq2seq，使用keras

[参考链接](https://zhuanlan.zhihu.com/p/40920384)

> seq2seq 是一个Encoder–Decoder 结构的网络，它的输入是一个序列，输出也是一个序列， Encoder 中将一个可变长度的信号序列变为固定长度的向量表达，Decoder 将这个固定长度的向量变成可变长度的目标的信号序列。

![](https://pic1.zhimg.com/80/v2-d90b7f4c45a9641296ce0862fe8ce0ac_hd.jpg)

想象一下翻译任务，input是一段英文，output是一段中文。

![](https://pic4.zhimg.com/80/v2-8ddf993a95ee6e525fe2cd5ccd49bba7_hd.jpg)

左侧为Encoder+输入，右侧为Decoder+输出。中间为Attention。

![](https://pic3.zhimg.com/80/v2-ea1fa8ee06cabc119c16ee0feafb5eaa_hd.jpg)

从左边Encoder开始，输入转换为word embedding, 进入LSTM。LSTM会在每一个时间点上输出hidden states。如图中的h1,h2,...,h8。

![](https://pic2.zhimg.com/80/v2-fcf2a510f1b7ff7f9e73d8140ec77f39_hd.jpg)

Decoder方面接受的是目标句子里单词的word embedding，和上一个时间点的hidden state。

Decoder的hidden state与Encoder所有的hidden states作为输入，放入Attention模块开始计算一个context vector。之后会介绍attention的计算方法。


![](https://pic1.zhimg.com/80/v2-73800a1528f84104d804f7d9a8951e24_hd.jpg)

来到时间点2，之前的context vector可以作为输入和目标的单词串起来作为lstm的输入。之后又回到一个hiddn state。以此循环。


![](https://pic1.zhimg.com/80/v2-bdf257055df55c4cc08bff527519f8cc_hd.jpg)

另一方面，context vector和decoder的hidden state合起来通过一系列非线性转换以及softmax最后计算出概率。


在luong中提到了三种score的计算方法。这里图解前两种：

![image.png](https://pic3.zhimg.com/80/v2-129287642af2e34d7e9e0afea9ae766e_hd.jpg)

**Attention score function: dot**

![](https://pic4.zhimg.com/80/v2-79f411f6569f9867bed489ff15415eaf_hd.jpg)

输入是encoder的所有hidden states H: 大小为(hid dim, sequence length)。decoder在一个时间点上的hidden state， s： 大小为（hid dim, 1）。

第一步：旋转H为（sequence length, hid dim) 与s做点乘得到一个 大小为(sequence length, 1)的分数。

第二步：对分数做softmax得到一个合为1的权重。

第三步：将H与第二步得到的权重做点乘得到一个大小为(hid dim, 1)的context vector。



**Attention score function: general**


![](https://pic1.zhimg.com/80/v2-4a57e750deb2c5c6f82040b88f05266c_hd.jpg)

输入是encoder的所有hidden states H: 大小为(hid dim1, sequence length)。decoder在一个时间点上的hidden state， s： 大小为（hid dim2, 1）。此处两个hidden state的纬度并不一样。

第一步：旋转H为（sequence length, hid dim1) 与 Wa [大小为 hid dim1, hid dim 2)] 做点乘， 再和s做点乘得到一个 大小为(sequence length, 1)的分数。

第二步：对分数做softmax得到一个合为1的权重。

第三步：将H与第二步得到的权重做点乘得到一个大小为(hid dim, 1)的context vector。

